In [1]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 10.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import sacrebleu
import time

In [3]:
en_file_path = "English.txt"
pr_file_path = "Persian.txt"

with open(en_file_path, "r") as file:
    english = [line.strip() for line in file]

with open(pr_file_path, "r") as file:
    persian = [line.strip() for line in file]

In [4]:
!pip install -U "transformers==4.40.0" --upgrade
!pip install accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 112.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  

In [5]:
import transformers
import torch

model_id = "unsloth/llama-3-8b-Instruct-bnb-4bit"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:159: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
def get_inference(prompt):
  messages = [
    {"role": "system", "content": ".تو یک دستیار ویرایشگر متن های فارسی هستی"},
    {"role": "user", "content": """{prompt}""".format(prompt = prompt)},
  ]

  prompt = pipeline.tokenizer.apply_chat_template(
          messages,
          tokenize=False,
          add_generation_prompt=True
  )

  terminators = [
      pipeline.tokenizer.eos_token_id,
      pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = pipeline(
      prompt,
      max_new_tokens=2000,
      eos_token_id=terminators,
      do_sample=False,
      temperature=0.7,
      top_p=0.9,
  )

  return(outputs[0]["generated_text"][len(prompt):])

**english to persian**

In [7]:

PROMPT_FEW = """
وظیفه‌ (تکلیفی) برای تو درنظر گرفته شده است، که مربوط به  ترجمه ماشینی است
به این منظور جمله‌ای به زبان انگلیسی به تو نشان داده می‌شود.
در نتیجه، شما باید آن جمله را به شکل مناسب و دقیق به زبان فارسی ترجمه کنید
مثال‌هایی از ترجمه‌ی جملات نمونه از زبان انگلیسی به زبان فارسی:
'I love programming.' -> 'من برنامه‌نویسی را دوست دارم.'
'Cooking a delicious meal requires a combination of ingredients and techniques.' -> 'پخت یک وعده غذای خوشمزه نیازمند ترکیبی از مواد اولیه و تکنیک‌هاست.'
'In the realm of artificial intelligence, neural networks play a pivotal role.' -> 'در دنیای هوش مصنوعی، شبکه‌های عصبی نقشی کلیدی ایفا می‌کنند.'


  فرمت خروجی:
  فرمت خروجی شما باید یک تاپل  باشد، که در آن هر تاپل از جمله انگلیسی و  ترجمه تشخیص داده شده تشکیل شده باشد.

حالا جمله زیر را ترجمه کن:
  """

In [14]:
len(persian)

200

In [13]:
import numpy as np

all_response = []
index = 0
for erow,prow in zip(english,persian):
    print('i: ',index)

    prompt_arman = f"""
    شرح وظیفه:
    {PROMPT_FEW}

     جمله انگلیسی:
    {erow}

    """
    response = get_inference(prompt_arman)
    index = index + 1
    all_response.append({'english':erow ,'persian':prow, 'response':response})
    if index%10==0 :
        np.save(f'MT_llama3_8b_fewshot.npy', np.array(all_response, dtype=object))


i:  0
i:  1
i:  2
i:  3
i:  4
i:  5
i:  6
i:  7
i:  8


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


i:  9
i:  10
i:  11
i:  12
i:  13
i:  14
i:  15
i:  16
i:  17
i:  18
i:  19
i:  20
i:  21
i:  22
i:  23
i:  24
i:  25
i:  26
i:  27
i:  28
i:  29
i:  30
i:  31
i:  32
i:  33
i:  34
i:  35
i:  36
i:  37
i:  38
i:  39
i:  40
i:  41
i:  42
i:  43
i:  44
i:  45
i:  46
i:  47
i:  48
i:  49
i:  50
i:  51
i:  52
i:  53
i:  54
i:  55
i:  56
i:  57
i:  58
i:  59
i:  60
i:  61
i:  62
i:  63
i:  64
i:  65
i:  66
i:  67
i:  68
i:  69
i:  70
i:  71
i:  72
i:  73
i:  74
i:  75
i:  76
i:  77
i:  78
i:  79
i:  80
i:  81
i:  82
i:  83
i:  84
i:  85
i:  86
i:  87
i:  88
i:  89
i:  90
i:  91
i:  92
i:  93
i:  94
i:  95
i:  96
i:  97
i:  98
i:  99
i:  100
i:  101
i:  102
i:  103
i:  104
i:  105
i:  106
i:  107
i:  108
i:  109
i:  110
i:  111
i:  112
i:  113
i:  114
i:  115
i:  116
i:  117
i:  118
i:  119
i:  120
i:  121
i:  122
i:  123
i:  124
i:  125
i:  126
i:  127
i:  128
i:  129
i:  130
i:  131
i:  132
i:  133
i:  134
i:  135
i:  136
i:  137
i:  138
i:  139
i:  140
i:  141
i:  142
i:  143
i:  144
i:  

**persian to english**

In [15]:

PROMPT_per_en = """
وظیفه‌ (تکلیفی) برای تو درنظر گرفته شده است، که مربوط به  ترجمه ماشینی است
به این منظور جمله‌ای به زبان فارسی به تو نشان داده می‌شود.
سپس تو باید آن جمله را به شکل مناسب و دقیق به زبان انگلیسی ترجمه کنی
مثال‌هایی از ترجمه‌ی جملات نمونه از زبان انگلیسی به زبان فارسی:
'من برنامه‌نویسی را دوست دارم.' ->  'I love programming.'
'پخت یک وعده غذای خوشمزه نیازمند ترکیبی از مواد اولیه و تکنیک‌هاست.' -> 'Cooking a delicious meal requires a combination of ingredients and techniques.'
'در دنیای هوش مصنوعی، شبکه‌های عصبی نقشی کلیدی ایفا می‌کنند.' -> 'In the realm of artificial intelligence, neural networks play a pivotal role.'


  فرمت خروجی:
  فرمت خروجی شما باید یک تاپل  باشد، که در آن هر تاپل از جمله انگلیسی و  ترجمه تشخیص داده شده تشکیل شده باشد.

حالا جمله زیر را ترجمه کن:
  """

In [16]:
import numpy as np

all_response = []
index = 0
for erow,prow in zip(english,persian):
    print('i: ',index)

    prompt = f"""
    شرح وظیفه:
    {PROMPT_per_en}

     جمله فارسی:
    {prow}

    """
    response = get_inference(prompt)
    index = index + 1
    all_response.append({'english':erow ,'persian':prow, 'response':response})
    if index%10==0 :
        np.save(f'MT_per_to_en_llama3_8b_fewshot.npy', np.array(all_response, dtype=object))


i:  0
i:  1
i:  2
i:  3
i:  4
i:  5
i:  6
i:  7
i:  8
i:  9
i:  10
i:  11
i:  12
i:  13
i:  14
i:  15
i:  16
i:  17
i:  18
i:  19
i:  20
i:  21
i:  22
i:  23
i:  24
i:  25
i:  26
i:  27
i:  28
i:  29
i:  30
i:  31
i:  32
i:  33
i:  34
i:  35
i:  36
i:  37
i:  38
i:  39
i:  40
i:  41
i:  42
i:  43
i:  44
i:  45
i:  46
i:  47
i:  48
i:  49
i:  50
i:  51
i:  52
i:  53
i:  54
i:  55
i:  56
i:  57
i:  58
i:  59
i:  60
i:  61
i:  62
i:  63
i:  64
i:  65
i:  66
i:  67
i:  68
i:  69
i:  70
i:  71
i:  72
i:  73
i:  74
i:  75
i:  76
i:  77
i:  78
i:  79
i:  80
i:  81
i:  82
i:  83
i:  84
i:  85
i:  86
i:  87
i:  88
i:  89
i:  90
i:  91
i:  92
i:  93
i:  94
i:  95
i:  96
i:  97
i:  98
i:  99
i:  100
i:  101
i:  102
i:  103
i:  104
i:  105
i:  106
i:  107
i:  108
i:  109
i:  110
i:  111
i:  112
i:  113
i:  114
i:  115
i:  116
i:  117
i:  118
i:  119
i:  120
i:  121
i:  122
i:  123
i:  124
i:  125
i:  126
i:  127
i:  128
i:  129
i:  130
i:  131
i:  132
i:  133
i:  134
i:  135
i:  136
i:  137
i:  13